In [10]:
import numpy as np
import torch
from scipy.integrate import odeint
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from torch import nn, autograd
from src.transformer_nn import TransformerModel
#from src.transformer_nn import SimpleLinearModel
from src.duffing_generator import DuffingGeneratorClass
from src.physics_loss import physics_loss_class
import torch.optim as op 
%load_ext jupyter_ai
import torch.optim as optim
#from torch.utils.data import Data
import transformers #import Transformer


The jupyter_ai extension is already loaded. To reload it, use:
  %reload_ext jupyter_ai


In [17]:
x = torch.linspace(0, 1, 1000).view(-1, 1)
dg = DuffingGeneratorClass()  # Instantiate your class

device = 'cpu'
torch.manual_seed(123)
physics_loss_instance = physics_loss_class()
# Set up the physics loss training locations
x_physics = torch.linspace(0, 1, 15).view(-1, 1).requires_grad_(True)
X_BOUNDARY = 0.0  # Example value, adjust as needed
F_BOUNDARY = 0.0  # Example value, adjust as needed
n_time_features = 1  # Assuming the time component is a single feature
n_param_features = 5  # Assuming there are 5 other parameter features
n_hidden = 128
n_layers = 8
n_heads = 4
# Initialize model, optimizer, and loss function
model = TransformerModel(n_time_features, n_param_features, n_hidden, n_layers, n_heads).to(device)
# Example: Learning rate scheduler

optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_instance = physics_loss_class()
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
num_epochs = 100  # Adjust as needed

In [18]:
'''# Create a small batch of test data with varying time
test_time = torch.linspace(0, 1, 10).view(-1, 1)  # 10 time steps
test_params = torch.zeros(10, 5)  # Keep other parameters constant
test_x_combined = torch.cat((test_time, test_params), dim=1).to(device)

# Get model's predictions for this test data
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_output = model(test_x_combined)
    print("Model output for test data:", test_output)'''

'# Create a small batch of test data with varying time\ntest_time = torch.linspace(0, 1, 10).view(-1, 1)  # 10 time steps\ntest_params = torch.zeros(10, 5)  # Keep other parameters constant\ntest_x_combined = torch.cat((test_time, test_params), dim=1).to(device)\n\n# Get model\'s predictions for this test data\nmodel.eval()  # Set the model to evaluation mode\nwith torch.no_grad():\n    test_output = model(test_x_combined)\n    print("Model output for test data:", test_output)'

In [ ]:


# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    num_batches = 100
    boundary_loss_weight = 0  # Adjust as necessary

    for batch_idx, batch in enumerate(dg.duffing_generator_batch(num_batches)):
        x_combined, y_batch = batch
        x_combined, y_batch = x_combined.to(device), y_batch.to(device)
    
        x_boundary = torch.tensor([[X_BOUNDARY]], dtype=torch.float32).to(device)
        target_value = torch.tensor([[F_BOUNDARY]], dtype=torch.float32).to(device)
    
        optimizer.zero_grad()
    
        # Slice out a small number of points and corresponding parameters
        indices = torch.arange(0, 10, 1000).to(device)  # Adjust the slicing as needed
        x_data = x_combined[indices, 0:1]
        params_data = x_combined[indices, 1:]
        y_data = y_batch[indices]
        # Compute physics loss
        x_time = x_combined[:, 0:1]
        x_params = x_combined[:, 1:]
    
        # Model prediction for data loss
        yh = model(x_data, params_data)
        mse_loss = torch.mean((yh - y_data) ** 2)
        def boundary_condition_loss(model, x_boundary, target_value):
            # Create a tensor of zeros with the correct shape for the parameters
            zero_params = torch.zeros_like(x_boundary).expand(-1, 5)  # Adjust '5' if the number of parameters is different
        
            model_output = model(x_boundary, x_params)
            return torch.mean((model_output - target_value) ** 2)
     
        # Compute boundary loss
        boundary_loss = boundary_condition_loss(model, x_boundary, target_value)
  
        loss_physics = loss_instance.physics_loss(model, x_time, x_params, device)
    
        # Combine losses
        total_loss = mse_loss + boundary_loss_weight * boundary_loss + loss_physics
    
        total_loss.backward()
        optimizer.step()
    
        print(f"Batch {batch_idx}, Total Loss: {total_loss.item()}")
    # Visualization every 10 epochs
    if (epoch+ 1) % 10 == 0:
        model.eval()  # Set the model to evaluation mode
        
        with torch.no_grad():
            yh = model(x_time, x_params).detach().cpu().numpy()
            time_points = x_combined[:, 0].cpu().numpy()  # Extract time points from combined input
            plt.figure(figsize=(10, 5))
            plt.plot(time_points, y_batch.cpu().numpy(), label='Ground Truth')
            plt.plot(time_points, yh, label='Neural Network Output')
            plt.scatter(time_points, y_batch.cpu().numpy(), color='red', label='Training Points')
            plt.xlabel("Time")
            plt.ylabel("Value")
            plt.legend()
            plt.title(f"Model Output vs Ground Truth at Epoch {epoch+1}")
            plt.show()

    # Optional: Add validation logic here

# Optional: Save the model after training
# torch.save(model.state_dict(), 'model.pth')

Batch 0, Total Loss: 0.007273682858794928
Batch 1, Total Loss: 0.027515480294823647
Batch 2, Total Loss: 0.0073003401048481464
Batch 3, Total Loss: 0.01609214022755623
Batch 4, Total Loss: 0.0074824439361691475
Batch 5, Total Loss: 0.02104845456779003
Batch 6, Total Loss: 0.013806751929223537
Batch 7, Total Loss: 0.020815791562199593
Batch 8, Total Loss: 0.016188398003578186
Batch 9, Total Loss: 0.011843899264931679
Batch 10, Total Loss: 7.949159044073895e-05
Batch 11, Total Loss: 0.003177050966769457
Batch 12, Total Loss: 0.011062638834118843
Batch 13, Total Loss: 0.00020531962218228728
Batch 14, Total Loss: 0.00016681004490237683
Batch 15, Total Loss: 0.002340924460440874
Batch 16, Total Loss: 0.0016657405067235231
Batch 17, Total Loss: 0.0002608431095723063
Batch 18, Total Loss: 0.019579026848077774
Batch 19, Total Loss: 0.0008842418901622295
Batch 20, Total Loss: 0.0001360338064841926
Batch 21, Total Loss: 6.24820459051989e-05
Batch 22, Total Loss: 0.015589752234518528
Batch 23, To

In [ ]:
print(torch.cuda.is_available())

In [ ]:
# Define the dimensions or use actual test data for test_input
test_input_dimensions = (100, 6)  # Example dimensions, modify as necessary
test_input = torch.randn(test_input_dimensions, dtype=torch.double, requires_grad=True)  # Create a random tensor with requires_grad set to True

# Convert test input to Float
test_input = test_input.float()

# Move model and test_input to the same device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
test_input = test_input.to(device)




# Test function for gradcheck
def test_func(input):
    return model(input)
# Perform the gradcheck with the modified test input
gradcheck_passed = gradcheck(test_func, test_input, eps=1e-5, atol=1e-4)
print("Gradcheck passed:", gradcheck_passed)

In [ ]:

# Create a sample input tensor with batch size greater than 1
sample_input = torch.randn(2, N_INPUT, requires_grad=True)

# Move model and sample_input to the same device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
sample_input = sample_input.to(device)

# Forward pass
output = model(sample_input)

# Sample loss computation (replace with a loss appropriate for your model)
sample_loss = output.sum()

# Backward pass
sample_loss.backward()

# Check if gradients are computed for the input
print("Gradients for the input tensor:", sample_input.grad)